In [1]:
import google.cloud.bigquery as bq
import pandas_gbq
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier

import itertools
import collections
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from scipy.stats import norm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
%matplotlib inline


In [ ]:
project_id = '.................'
df = pd.read_gbq("""with call_list as (
	(
		select
			*
		from
			ad_hoc.store_conversions
		where
			converted = 1
		order by rand()
		limit 358
	)
	union all
	(
		select
			*
		from
			ad_hoc.store_conversions
		where
			converted = 0
		order by rand()
		limit 358
	)
),
live_ads as (
	select
		case
			when ac_store_id.ad_id is not null then cast(ac_store_id.value as int64)
			else ads.store_id
			end as store_id,
		aa.date,
		count(*) live_ads,
		sum(
			case
				when case when ac_type.ad_id is not null then ac_type.value else ads.type end in ('let', 'rent') then coalesce(
					case
						when ac_monthly_rent.ad_id is not null then cast(ac_monthly_rent.value as int64)
						else cast(ap_monthly_rent.value as int64)
						end,
					case
						when ac_max_rent.ad_id is not null then cast(ac_max_rent.value as int64)
						else cast(ap_max_rent.value as int64)
						end,
					case
						when ac_rental.ad_id is not null then cast(ac_rental.value as int64)
						else cast(ap_rental.value as int64)
						end
					)
				else
					case
						when ac_price.ad_id is not null then cast(ac_price.value as int64)
						else ads.price
						end
				end
			)/count(*) as avg_price_per_live_ad
	from
		blocket_tf.active_ads aa
	join blocket.ads ads using (ad_id)
	left join blocket_tf.ad_changes ac_store_id on
		ac_store_id.ad_id = aa.ad_id
		and ac_store_id.column_name = 'store_id'
		and not ac_store_id.is_param
		and cast(aa.date + 1 as timestamp) >= ac_store_id.start_time
		and cast(aa.date + 1 as timestamp) < ac_store_id.end_time
	left join blocket_tf.ad_changes ac_price on
		ac_price.ad_id = aa.ad_id
		and ac_price.column_name = 'price'
		and not ac_price.is_param
		and cast(aa.date + 1 as timestamp) >= ac_price.start_time
		and cast(aa.date + 1 as timestamp) < ac_price.end_time
	left join blocket_tf.ad_changes ac_type on
		ac_type.ad_id = aa.ad_id
		and ac_type.column_name = 'type'
		and not ac_type.is_param
		and cast(aa.date + 1 as timestamp) >= ac_type.start_time
		and cast(aa.date + 1 as timestamp) < ac_type.end_time
	left join blocket.ad_params ap_monthly_rent on
		ap_monthly_rent.ad_id = aa.ad_id
		and ap_monthly_rent.name = 'monthly_rent'
	left join blocket_tf.ad_changes ac_monthly_rent on
		ac_monthly_rent.ad_id = aa.ad_id
		and ac_monthly_rent.column_name = 'monthly_rent'
		and ac_monthly_rent.is_param
		and cast(aa.date + 1 as timestamp) >= ac_monthly_rent.start_time
		and cast(aa.date + 1 as timestamp) < ac_monthly_rent.end_time
	left join blocket.ad_params ap_max_rent on
		ap_max_rent.ad_id = aa.ad_id
		and ap_max_rent.name = 'max_rent'
	left join blocket_tf.ad_changes ac_max_rent on
		ac_max_rent.ad_id = aa.ad_id
		and ac_max_rent.column_name = 'max_rent'
		and ac_max_rent.is_param
		and cast(aa.date + 1 as timestamp) >= ac_max_rent.start_time
		and cast(aa.date + 1 as timestamp) < ac_max_rent.end_time
	left join blocket.ad_params ap_rental on
		ap_rental.ad_id = aa.ad_id
		and ap_rental.name = 'rental'
	left join blocket_tf.ad_changes ac_rental on
		ac_rental.ad_id = aa.ad_id
		and ac_rental.column_name = 'rental'
		and ac_rental.is_param
		and cast(aa.date + 1 as timestamp) >= ac_rental.start_time
		and cast(aa.date + 1 as timestamp) < ac_rental.end_time
	group by 1,2
	having store_id is not null
),
credit_balance as (
	select distinct
		store_id,
		date,
		sum(coalesce(coin, 0)) over (partition by store_id order by date rows unbounded preceding) as balance
	from
		(
			select
				store_id,
				date(timestamp) as date,
				sum(coalesce(coin, 0)) as coin
			from
				visualization.unutilized_credits
			group by 1,2
		)
)
select
	cl.store_id,
	cl.call_date,
	cast(s.category as string) as store_category,
	cast(s.region as string) as store_region,
	coalesce(cb.balance, 0) as credit_balance,
	coalesce(la.live_ads, 0) as live_ads,
	coalesce(la.avg_price_per_live_ad, 0) as avg_price_per_live_ad,
	sum(case when fd.product = 'paid' then coin else 0 end) as credit_purchased,
	sum(case when fd.product = 'expired' then coin else 0 end) as credit_expired,
	sum(case when fd.product in ('listing', 'prolong') then coin else 0 end) as credit_listing,
	sum(case when fd.product not in ('listing', 'prolong', 'paid', 'expired') then coin else 0 end) as credit_ps,
	cl.converted
from
	call_list cl
join blocket.stores s using (store_id)
left join live_ads la on
	la.store_id = cl.store_id
	and la.date = cl.call_date - 1
left join revenue.fifo_detailed fd on
	fd.store_id = cl.store_id
	and fd.product not in ('void', 'refund_admin', 'compliment', 'transfer_in', 'transfer_out', 'edit')
	and fd.date between cl.call_date - 91 and cl.call_date - 1
left join credit_balance cb on
	cb.store_id = cl.store_id
	and cb.date = cl.call_date - 1
group by 1,2,3,4,5,6,7, 12""",              
                 project_id = project_id)

In [ ]:
df.describe()

In [ ]:
df1 = df.copy()

df1=df.copy()

In [ ]:
df1 =df1.drop('call_date',axis=1)

In [ ]:
df1 =df1.drop('store_id',axis=1)


In [ ]:
df1 =df1.drop('store_category',axis=1)


In [ ]:
df1 =df1.drop('store_region',axis=1)


In [ ]:
df2 = df1.copy()
df2.info()

In [ ]:
(len(df2.loc[df2.converted==1])) / (len(df2.loc[df2.converted == 0])) * 100

In [ ]:
X = df2.drop('converted', axis=1)
y = df2['converted']
rus = RandomUnderSampler(random_state=42) 
#Under-samples the majority classes by randomly picking samples with or without replacement.
X_rs, y_rs = rus.fit_resample(X,y) 
#fit - Finds for the classes statistics before performing sampling.

In [ ]:
df_rs = pd.DataFrame(np.hstack((X_rs,y_rs[:, None])), columns=df2.columns)
df_rs.converted = df_rs.converted.astype(int)

In [ ]:
def naive_predictor(df2): ## gives better accuracy for majority classes
    TP = df2.converted.count() - np.sum(df2.converted)
    FP = np.sum(df2.converted)
    TN = 0
    FN = 0
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    recall = TP/(TP+FN)
    precision = TP/(TP+FP)
    fscore = (2*precision*recall)/(precision + recall)
    print("[Accuracy score: {:.4f}, precision: {:.4f}, recall: {:.4f}, f1-score: {:.4f}]".format(accuracy, precision, recall, fscore))    
print('Naive predictor for original dataset:')
naive_predictor(df2)
print('-'*100)
print('Naive predictor for undersampled dataset:')
naive_predictor(df_rs)

In [ ]:
# RANDOM UNDERSAMPLING METHOD

In [ ]:
X = df_rs.drop('converted', axis=1)
y = df_rs['converted']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
### CROSS VALIDATION
classifiers = {
    "Logistic Regression": LogisticRegression(),
#     "Support Vector Classifier": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "KNearest": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "MLP": MLPClassifier(),
    "Ridge": RidgeClassifier(),
    "Stochastic Gradient Boosting": GradientBoostingClassifier()
    
}

In [ ]:
print('Cross-Validation Scores:-')
for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    cv_score = cross_val_score(classifier, X_train, y_train, cv=5) 
    #number of cv you have to try for each selected set of hyperparameters
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 2)))

In [ ]:
params = {
    "Logistic Regression": {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
#     "Support Vector Classifier": {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
    "Decision Tree": {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))},
    "KNearest": {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},
    "Random Forest": {'n_estimators': [10, 100, 1000],'max_features':['auto', 'sqrt']},
    "MLP": {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]},
    "Ridge": {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]},
    "Stochastic Gradient Boosting": {'n_estimators':range(20,81,10),'max_features':['sqrt']}
}

In [ ]:
def gridsearch(classifier, params):
    grid_classifier = GridSearchCV(classifier, params)
    grid_classifier.fit(X_train, y_train)
    best_classifier = grid_classifier.best_estimator_
    return best_classifier

In [ ]:
print('Cross-Validation Scores after applying GridSearch -')
for key, classifier in classifiers.items():
    classifier = gridsearch(classifier,params[key])
    cv_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 2)))

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    ####Learning curve ####### To check if there's any overfitting
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(name='Training score - Standard Deviation',x=train_sizes,y=train_scores_mean+train_scores_std,
                            mode='lines',showlegend=False,marker=dict(color='blue')))
    
    fig.add_trace(go.Scatter(name='Training score', x=train_sizes, y=train_scores_mean, fill='tonexty',mode='lines+markers',
                            marker=dict(color='blue')))
    
    fig.add_trace(go.Scatter(name='Training score + Standard Deviation', x=train_sizes, y=train_scores_mean-train_scores_std,
                            mode='lines', fill='tonexty', showlegend=False,marker=dict(color='blue')))
    
    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean+test_scores_std,mode='lines',showlegend=False,
                             marker=dict(color='orange')))
   
    fig.add_trace(go.Scatter(name='Validation Score',x=train_sizes, y=test_scores_mean, mode='lines+markers',fill='tonexty',
                            marker=dict(color='orange')))

    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean-test_scores_std,mode='lines',fill='tonexty',showlegend=False,
                            marker=dict(color='orange')))

    fig.update_layout(width=700,height=400,template='seaborn',title=title,margin=dict(l=60,r=0,b=0,t=40),legend=dict(orientation='h',x=0.5,y=1),
                        xaxis=dict(title='Training examples',mirror=True,linecolor='black',linewidth=2),
                        yaxis=dict(title='Scores',range=ylim if ylim is not None else None,
                        mirror=True,linecolor='black',linewidth=2))
    return fig

In [ ]:
n_jobs = 10
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
log_reg = gridsearch(LogisticRegression(), params['Logistic Regression'])
# svc = gridsearch(SVC(), params['Support Vector Classifier'])
decision_tree = gridsearch(DecisionTreeClassifier(), params['Decision Tree'])
knearest = gridsearch(KNeighborsClassifier(), params['KNearest'])
random_forest = gridsearch(RandomForestClassifier(), params['Random Forest'])
MLP = gridsearch(MLPClassifier(),params['MLP'])
Ridge = gridsearch(RidgeClassifier(), params['Ridge'])
Stochastic_gradient_boosting = gridsearch(GradientBoostingClassifier(), params['Stochastic Gradient Boosting'])

In [ ]:
plot_learning_curve(log_reg,'Logistic Regression', X_train, y_train, n_jobs=10)

In [ ]:
# plot_learning_curve(svc, 'Support Vector Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(decision_tree, 'Decision Tree Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(knearest, 'KNearest Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(random_forest, 'Random Forest Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(MLP, 'MLP Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(Ridge, 'Ridge Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(Stochastic_gradient_boosting, 'Stochastic Gradient Boosting Classifier', X_train, y_train, n_jobs=10)

In [ ]:
###### KNN AND DT IS CLOSER TO IDEAL CASE ( closer to value '1')

In [ ]:
log_pred = log_reg.predict(X_test)
# svc_pred = svc.predict(X_test)
tree_pred = decision_tree.predict(X_test)
knear_pred = knearest.predict(X_test)
rfnear_pred = random_forest.predict(X_test)
mlpnear_pred = MLP.predict(X_test)
Rnear_pred = Ridge.predict(X_test)
SGBnear_pred = Stochastic_gradient_boosting.predict(X_test)

log_fpr, log_tpr, log_threshold = roc_curve(y_test, log_pred)
# svc_fpr, svc_tpr, svc_threshold = roc_curve(y_test, svc_pred)
tree_fpr, tree_tpr, tree_threshold = roc_curve(y_test, tree_pred)
knear_fpr, knear_tpr, knear_threshold = roc_curve(y_test, knear_pred)
rfnear_fpr, rfnear_tpr, rfnear_threshold = roc_curve(y_test, rfnear_pred)
mlpnear_fpr, mlpnear_tpr, mlpnear_threshold = roc_curve(y_test, mlpnear_pred)
Rnear_fpr, Rnear_tpr, Rnear_threshold = roc_curve(y_test, Rnear_pred)
SGBnear_fpr, SGBnear_tpr, SGBnear_threshold = roc_curve(y_test, SGBnear_pred)

fig = go.Figure()
fig.add_trace(go.Scatter(name='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_test, log_pred)),x=log_fpr,y=log_tpr,mode='lines'))
# fig.add_trace(go.Scatter(name='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_test, svc_pred)),x=svc_fpr,y=svc_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_test, tree_pred)),x=tree_fpr,y=tree_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='K-Nearest Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_test, knear_pred)),x=knear_fpr,y=knear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Random Forest Classifier Score: {:.4f}'.format(roc_auc_score(y_test, rfnear_pred)),x=rfnear_fpr,y=rfnear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='MLP Classifier Score: {:.4f}'.format(roc_auc_score(y_test, mlpnear_pred)),x=mlpnear_fpr,y=mlpnear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Ridge Classifier Score: {:.4f}'.format(roc_auc_score(y_test, Rnear_pred)),x=Rnear_fpr,y=Rnear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Stochastic Gradient Classifier Score: {:.4f}'.format(roc_auc_score(y_test, SGBnear_pred)),x=SGBnear_fpr,y=SGBnear_tpr,mode='lines'))

fig.add_trace(go.Scatter(name='AUC-ROC=0.5',x=[0,1],y=[0,1],line=dict(dash='dot'),showlegend=False))
fig.update_layout(width=1000,height=600,xaxis=dict(mirror=True,linewidth=2,linecolor='black'),yaxis=dict(mirror=True,linewidth=2,linecolor='black'),
            title='ROC Curve<br>(All Classifiers)',template='seaborn',legend=dict(x=0.46,y=0,traceorder="normal", font=dict(family="sans-serif",
            size=12,color="black"),bgcolor="Lightgray",bordercolor="Black",borderwidth=2),
            annotations=[dict(x=0.5,y=0.5,xref="x", yref="y",text="Minimum ROC Score of 50% <br>",showarrow=True,arrowhead=7,ax=40,ay=50)])
fig.show()

In [ ]:
# From the above curve it can be seen that DT has the highest ROC curve SO, 
# it is the best model according to this metric.

In [ ]:
labels = ['Non-Converted', 'Converted']
print('#'*125)
print('Logistic Regression:')
print(classification_report(y_test, log_pred, target_names=labels))
print('#'*125)
# print('Support Vector Classifier:')
# print(classification_report(y_test, svc_pred, target_names=labels))
# print('#'*125)
print('Decision Tree:')
print(classification_report(y_test, tree_pred, target_names=labels))
print('#'*125)
print('KNearest Neighbours:')
print(classification_report(y_test, knear_pred, target_names=labels))
print('#'*125)
print('Random Forests:')
print(classification_report(y_test, rfnear_pred, target_names=labels))
print('#'*125)
print('MLP:')
print(classification_report(y_test, mlpnear_pred, target_names=labels))
print('#'*125)
print('Ridge:')
print(classification_report(y_test, Rnear_pred, target_names=labels))
print('#'*125)
print('Stochastic Gradient Boosting:')
print(classification_report(y_test, SGBnear_pred, target_names=labels))

In [ ]:
print('-'*65)
print('Ridge Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,Rnear_pred))
print('Accuracy score:',accuracy_score(y_test, Rnear_pred))
print('F1 score:',f1_score(y_test, Rnear_pred))
print('-'*65)

print('-'*65)
print('Gradient Boosting Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,SGBnear_pred))
print('Accuracy score:',accuracy_score(y_test, SGBnear_pred))
print('F1 score:',f1_score(y_test, SGBnear_pred))
print('-'*65)

print('-'*65)
print('MLP Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,mlpnear_pred))
print('Accuracy score:',accuracy_score(y_test, mlpnear_pred))
print('F1 score:',f1_score(y_test, mlpnear_pred))
print('-'*65)

print('-'*65)
print('Decision Tree Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,tree_pred))
print('Accuracy score:',accuracy_score(y_test, tree_pred))
print('F1 score:',f1_score(y_test, tree_pred))
print('-'*65)

print('-'*65)
print('K-nearest Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,knear_pred))
print('Accuracy score:',accuracy_score(y_test, knear_pred))
print('F1 score:',f1_score(y_test, knear_pred))
print('-'*65)

print('-'*65)
print('Random Forest Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,rfnear_pred))
print('Accuracy score:',accuracy_score(y_test, rfnear_pred))
print('F1 score:',f1_score(y_test, rfnear_pred))
print('Recall:',recall_score(y_test, rfnear_pred))
print('Precision:',precision_score(y_test, rfnear_pred))

print('-'*65)

print('-'*65)
print('Logistic Regression Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,log_pred))
print('Accuracy score:',accuracy_score(y_test, log_pred))
print('F1 score:',f1_score(y_test, log_pred))
print('-'*65)
